STM我突然发现性别那里很多0值，这不搞笑么。。有75个，没必要单独建模或者插值。直接全删了。果然先进行完全的数据探索是必要的。

In [22]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import os

from empirical_formulas import apply_formulas, func_name_list


In [3]:
df = pd.read_excel('GFR-SVR数据.xls')
df.columns = ['id', 'age', 'sex', 'rGFR', 'Scr', 'Cys']
df.shape

(1197, 6)

In [4]:
df['Cys'] = pd.to_numeric(df['Cys'], errors='coerce')
df=df.dropna()
df.shape

(1188, 6)

In [15]:
df = df[(18<df['age']) & (df['age']<=100) & (1<=df['sex']) & (df['sex']<=2) & (5 <= df['rGFR']) & (df['rGFR'] <= 150) & \
       (0.0 <= df['Scr']) & (df['Scr'] <= 3000) & (0.2 <= df['Cys']) & (df['Cys'] < 5.0)]
df.shape

(1081, 6)

In [20]:
apply_formulas(df)

In [23]:
(df[func_name_list].apply(lambda col:col-df['rGFR'],axis=0)**2).mean(axis=0) #均方误差

CKD_EPI_Cr        3.474546e+02
CKD_EPI_Cys       4.016079e+02
CKD_EPI_Cr_Cys    2.998761e+02
FAS_cr            4.791257e+02
FAS_Cys           1.723455e+06
FAS_Cr_Cys        1.943686e+06
dtype: float64

In [26]:
df.groupby('sex').mean()

,id,age,rGFR,Scr,Cys,CKD_EPI_Cr,CKD_EPI_Cys,CKD_EPI_Cr_Cys,FAS_cr,FAS_Cys,FAS_Cr_Cys
sex,,,,,,,,,,,
1,685.293103,55.586207,63.802748,1.813219,1.693448,66.166964,57.482709,60.898614,67.133613,1047.591832,1110.852965
2,503.615584,54.862338,71.284883,1.327743,1.525481,72.562213,62.007541,66.497312,73.459845,1065.895221,1108.766670


我当初要执行了这个例行公事就不至于连0的存在都尼玛没发现了。

In [30]:
dfp = df.copy()

yp = np.log(dfp['rGFR'])
Xp = np.stack([dfp['age'], np.log(dfp['Scr']), np.log(dfp['Cys'])], 1)

Xp_sex = np.stack([dfp['age'], np.log(dfp['Scr']), np.log(dfp['Cys']), dfp['sex']], 1)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(Xp_sex, yp, test_size=0.2, random_state=51)

In [32]:
mat = np.empty([3,3,3,2])
for i,gamma in enumerate([0.05,0.1,0.15]):
    for j,C in enumerate([500,1e3,1500]):
        for k,epsilon in enumerate([0.05,0.1,0.15]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((np.exp(y_train) - np.exp(y_train_pred))**2)
            test_loss = np.mean((np.exp(y_test) - np.exp(y_test_pred))**2)
            
            print('gamma = {} C = {} epsilon = {} train loss ={} test loss = {}'.format(gamma, C, epsilon, train_loss, test_loss))
            mat[i,j,k,0] = train_loss
            mat[i,j,k,1] = test_loss

gamma = 0.05 C = 500 epsilon = 0.05 train loss =162.7287532126021 test loss = 279.14374322074394
gamma = 0.05 C = 500 epsilon = 0.1 train loss =161.58430495529606 test loss = 278.62134321428994
gamma = 0.05 C = 500 epsilon = 0.15 train loss =166.7621744264278 test loss = 255.40518305999345
gamma = 0.05 C = 1000.0 epsilon = 0.05 train loss =159.05065096933146 test loss = 354.33336193454693
gamma = 0.05 C = 1000.0 epsilon = 0.1 train loss =158.413402752551 test loss = 395.2911787856813
gamma = 0.05 C = 1000.0 epsilon = 0.15 train loss =162.64756314507082 test loss = 277.5286203966757
gamma = 0.05 C = 1500 epsilon = 0.05 train loss =156.74274954130422 test loss = 465.46008336766505
gamma = 0.05 C = 1500 epsilon = 0.1 train loss =156.30058851918227 test loss = 439.24834729329274
gamma = 0.05 C = 1500 epsilon = 0.15 train loss =160.52438249825488 test loss = 299.0835570419334
gamma = 0.1 C = 500 epsilon = 0.05 train loss =127.83022491932736 test loss = 400.19842945899137
gamma = 0.1 C = 500

In [33]:
mat2 = np.empty([3,3,3,2])
for i,gamma in enumerate([0.025,0.05,0.075]):
    for j,C in enumerate([500,750,1000]):
        for k,epsilon in enumerate([0.05,0.1,0.15]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((np.exp(y_train) - np.exp(y_train_pred))**2)
            test_loss = np.mean((np.exp(y_test) - np.exp(y_test_pred))**2)
            
            print('gamma = {} C = {} epsilon = {} train loss ={} test loss = {}'.format(gamma, C, epsilon, train_loss, test_loss))
            mat2[i,j,k,0] = train_loss
            mat2[i,j,k,1] = test_loss

gamma = 0.025 C = 500 epsilon = 0.05 train loss =173.5309195412155 test loss = 207.95502432712797
gamma = 0.025 C = 500 epsilon = 0.1 train loss =172.50613505887026 test loss = 211.9408166617084
gamma = 0.025 C = 500 epsilon = 0.15 train loss =174.0490911942909 test loss = 205.21566560971112
gamma = 0.025 C = 750 epsilon = 0.05 train loss =172.36480133868304 test loss = 215.403949863005
gamma = 0.025 C = 750 epsilon = 0.1 train loss =170.7379119551372 test loss = 215.2239868741341
gamma = 0.025 C = 750 epsilon = 0.15 train loss =173.04364540279832 test loss = 210.7536281993375
gamma = 0.025 C = 1000 epsilon = 0.05 train loss =170.81837622478648 test loss = 218.18168387453844
gamma = 0.025 C = 1000 epsilon = 0.1 train loss =169.92360651965026 test loss = 218.70983529096858
gamma = 0.025 C = 1000 epsilon = 0.15 train loss =172.66702056133144 test loss = 213.79445915077264
gamma = 0.05 C = 500 epsilon = 0.05 train loss =162.7287532126021 test loss = 279.14374322074394
gamma = 0.05 C = 500

In [34]:
mat3 = np.empty([3,3,3,2])
for i,gamma in enumerate([0.01,0.02,0.03]):
    for j,C in enumerate([500,750,1000]):
        for k,epsilon in enumerate([0.05,0.1,0.15]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((np.exp(y_train) - np.exp(y_train_pred))**2)
            test_loss = np.mean((np.exp(y_test) - np.exp(y_test_pred))**2)
            
            print('gamma = {} C = {} epsilon = {} train loss ={} test loss = {}'.format(gamma, C, epsilon, train_loss, test_loss))
            mat3[i,j,k,0] = train_loss
            mat3[i,j,k,1] = test_loss

gamma = 0.01 C = 500 epsilon = 0.05 train loss =187.37402101973282 test loss = 196.86518478381203
gamma = 0.01 C = 500 epsilon = 0.1 train loss =188.6801103761815 test loss = 202.28830446729643
gamma = 0.01 C = 500 epsilon = 0.15 train loss =190.930053705017 test loss = 206.0921363265256
gamma = 0.01 C = 750 epsilon = 0.05 train loss =185.4375414134833 test loss = 196.11903532814492
gamma = 0.01 C = 750 epsilon = 0.1 train loss =187.74394653487158 test loss = 203.5078878350234
gamma = 0.01 C = 750 epsilon = 0.15 train loss =189.3201214042461 test loss = 204.67946095122016
gamma = 0.01 C = 1000 epsilon = 0.05 train loss =184.5098443082765 test loss = 197.09204040406945
gamma = 0.01 C = 1000 epsilon = 0.1 train loss =187.21849883585128 test loss = 204.31265608720156
gamma = 0.01 C = 1000 epsilon = 0.15 train loss =188.0000255844576 test loss = 204.45387992470972
gamma = 0.02 C = 500 epsilon = 0.05 train loss =179.5519276204166 test loss = 206.36685300230522
gamma = 0.02 C = 500 epsilon =

In [35]:
mat3 = np.empty([3,3,3,2])
for i,gamma in enumerate([0.00001,0.006,0.012]):
    for j,C in enumerate([500,750,1000]):
        for k,epsilon in enumerate([0.05,0.1,0.15]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((np.exp(y_train) - np.exp(y_train_pred))**2)
            test_loss = np.mean((np.exp(y_test) - np.exp(y_test_pred))**2)
            
            print('gamma = {} C = {} epsilon = {} train loss ={} test loss = {}'.format(gamma, C, epsilon, train_loss, test_loss))
            mat3[i,j,k,0] = train_loss
            mat3[i,j,k,1] = test_loss

gamma = 1e-05 C = 500 epsilon = 0.05 train loss =205.7840238324065 test loss = 192.505546064265
gamma = 1e-05 C = 500 epsilon = 0.1 train loss =207.3699290258895 test loss = 194.1446823419283
gamma = 1e-05 C = 500 epsilon = 0.15 train loss =208.39508910215847 test loss = 195.52176982890728
gamma = 1e-05 C = 750 epsilon = 0.05 train loss =205.33640430430128 test loss = 193.44650232794228
gamma = 1e-05 C = 750 epsilon = 0.1 train loss =206.32210100949055 test loss = 194.6831347149918
gamma = 1e-05 C = 750 epsilon = 0.15 train loss =207.60071190985502 test loss = 196.07520683096072
gamma = 1e-05 C = 1000 epsilon = 0.05 train loss =205.28187905279253 test loss = 194.02685258755878
gamma = 1e-05 C = 1000 epsilon = 0.1 train loss =205.96919970609193 test loss = 194.98073489309482
gamma = 1e-05 C = 1000 epsilon = 0.15 train loss =207.3756040582272 test loss = 196.6080300236742
gamma = 0.006 C = 500 epsilon = 0.05 train loss =191.9343065226047 test loss = 197.1674352929921
gamma = 0.006 C = 50

In [36]:
mat3 = np.empty([3,3,3,2])
for i,gamma in enumerate([0.005,0.006,0.007]):
    for j,C in enumerate([250, 500,750]):
        for k,epsilon in enumerate([0.075,0.1,0.125]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((np.exp(y_train) - np.exp(y_train_pred))**2)
            test_loss = np.mean((np.exp(y_test) - np.exp(y_test_pred))**2)
            
            print('gamma = {} C = {} epsilon = {} train loss ={} test loss = {}'.format(gamma, C, epsilon, train_loss, test_loss))
            mat3[i,j,k,0] = train_loss
            mat3[i,j,k,1] = test_loss

gamma = 0.005 C = 250 epsilon = 0.075 train loss =195.2229429422545 test loss = 197.4751383242691
gamma = 0.005 C = 250 epsilon = 0.1 train loss =194.97076998275236 test loss = 198.02599766035885
gamma = 0.005 C = 250 epsilon = 0.125 train loss =195.52313700965877 test loss = 201.56784415855216
gamma = 0.005 C = 500 epsilon = 0.075 train loss =194.28294253086955 test loss = 197.90353591781556
gamma = 0.005 C = 500 epsilon = 0.1 train loss =194.79012458444618 test loss = 200.35451000499995
gamma = 0.005 C = 500 epsilon = 0.125 train loss =194.25364558965813 test loss = 201.15959198172186
gamma = 0.005 C = 750 epsilon = 0.075 train loss =193.83816197118483 test loss = 200.21466359583815
gamma = 0.005 C = 750 epsilon = 0.1 train loss =194.01911717293055 test loss = 198.4020863804242
gamma = 0.005 C = 750 epsilon = 0.125 train loss =193.96284692236168 test loss = 200.64420491496966
gamma = 0.006 C = 250 epsilon = 0.075 train loss =194.17662605827132 test loss = 200.404888260327
gamma = 0.0

In [43]:
1e-05 == 0.00001

True

目前最好的是
```
gamma = 1e-05 C = 500 epsilon = 0.05 train loss =205.7840238324065 test loss = 192.505546064265
```

In [46]:
mat3 = np.empty([3,3,3,2])
for i,gamma in enumerate([0.000001,0.00001,0.00002]):
    for j,C in enumerate([400, 500,600]):
        for k,epsilon in enumerate([0.025,0.05,0.075]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((np.exp(y_train) - np.exp(y_train_pred))**2)
            test_loss = np.mean((np.exp(y_test) - np.exp(y_test_pred))**2)
            
            print('gamma = {} C = {} epsilon = {} train loss ={} test loss = {}'.format(gamma, C, epsilon, train_loss, test_loss))
            mat3[i,j,k,0] = train_loss
            mat3[i,j,k,1] = test_loss

gamma = 1e-06 C = 400 epsilon = 0.025 train loss =306.9827990484481 test loss = 263.962156639115
gamma = 1e-06 C = 400 epsilon = 0.05 train loss =307.93915782868413 test loss = 264.5779388166863
gamma = 1e-06 C = 400 epsilon = 0.075 train loss =309.58836372018226 test loss = 266.0672393948384
gamma = 1e-06 C = 500 epsilon = 0.025 train loss =280.9184034707988 test loss = 243.38899262329917
gamma = 1e-06 C = 500 epsilon = 0.05 train loss =284.6917487215257 test loss = 246.33895630159438
gamma = 1e-06 C = 500 epsilon = 0.075 train loss =283.99516937063083 test loss = 246.697428561091
gamma = 1e-06 C = 600 epsilon = 0.025 train loss =265.9778585867699 test loss = 232.10206136297046
gamma = 1e-06 C = 600 epsilon = 0.05 train loss =264.94516260895455 test loss = 231.51552248160868
gamma = 1e-06 C = 600 epsilon = 0.075 train loss =267.2165446653172 test loss = 233.4360453941598
gamma = 1e-05 C = 400 epsilon = 0.025 train loss =206.21408759615701 test loss = 191.72241089410926
gamma = 1e-05 C

gamma = 1e-05 C = 400 epsilon = 0.025 train loss =206.21408759615701 test loss = 191.72241089410926